In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime

# Configurar estilo para mejores visualizaciones
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("viridis")

# Crear directorio de salida para gráficos si no existe
output_dir = 'eda_plots'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# 1. Cargar los datos
print("Cargando dataset...")
try:
    datos = pd.read_csv('campania_marketing_dataset.csv')
    print(f"Dataset cargado exitosamente con {datos.shape[0]} filas y {datos.shape[1]} columnas.")
except Exception as e:
    print(f"Error al cargar el dataset: {e}")
    exit(1)

# 2. Análisis inicial de los datos
print("\n=== VISIÓN GENERAL DE LOS DATOS ===")
print("\nPrimeras 5 filas del dataset:")
print(datos.head())

print("\nInformación del dataset:")
datos.info()

print("\nEstadísticas descriptivas:")
print(datos.describe().round(2))

# 3. Limpieza y preparación de datos
print("\n=== LIMPIEZA DE DATOS ===")

# Revisar duplicados
duplicados = datos.duplicated().sum()
print(f"\nNúmero de filas duplicadas: {duplicados}")
if duplicados > 0:
    datos = datos.drop_duplicates()
    print(f"Se eliminaron {duplicados} filas duplicadas.")

# Revisar valores faltantes
print("\nValores faltantes por columna:")
valores_faltantes = datos.isnull().sum()
print(valores_faltantes[valores_faltantes > 0])

# Calcular porcentaje de valores faltantes
porcentaje_faltantes = (datos.isnull().mean() * 100).round(2)
print("\nPorcentaje de valores faltantes por columna:")
print(porcentaje_faltantes[porcentaje_faltantes > 0])

# 4. Análisis univariante
print("\n=== ANÁLISIS UNIVARIANTE ===")

def analisis_variables_numericas(data, guardar_graficos=True):
    """Analiza variables numéricas con visualizaciones mejoradas"""
    # Seleccionar columnas numéricas
    columnas_numericas = data.select_dtypes(include=[np.number]).columns
    
    print(f"\nAnalizando {len(columnas_numericas)} variables numéricas...")
    
    for col in columnas_numericas:
        # Saltar columnas con demasiados valores únicos o ids
        if data[col].nunique() > data.shape[0] * 0.9 or 'id' in col.lower():
            print(f"Saltando {col} - probablemente una columna ID")
            continue
            
        print(f"\nAnálisis para {col}:")
        
        # Estadísticas básicas con detección de valores atípicos
        Q1 = data[col].quantile(0.25)
        Q3 = data[col].quantile(0.75)
        IQR = Q3 - Q1
        limite_inferior = Q1 - 1.5 * IQR
        limite_superior = Q3 + 1.5 * IQR
        atipicos = data[(data[col] < limite_inferior) | (data[col] > limite_superior)][col]
        
        datos_estadisticos = {
            'Media': data[col].mean(),
            'Mediana': data[col].median(),
            'Desv. Estándar': data[col].std(),
            'Mínimo': data[col].min(),
            'Máximo': data[col].max(),
            'Asimetría': data[col].skew(),
            'Curtosis': data[col].kurtosis(),
            'Cantidad Atípicos': len(atipicos),
            'Porcentaje Atípicos': (len(atipicos) / len(data[col])) * 100
        }
        
        for stat, value in datos_estadisticos.items():
            print(f"{stat}: {value:.4f}")
        
        if guardar_graficos:
            fig, axes = plt.subplots(1, 2, figsize=(15, 6))
            
            # Histograma con KDE
            sns.histplot(data[col], kde=True, ax=axes[0])
            axes[0].set_title(f'Distribución de {col}')
            axes[0].set_xlabel(col)
            axes[0].set_ylabel('Frecuencia')
            
            # Box plot
            sns.boxplot(y=data[col], ax=axes[1])
            axes[1].set_title(f'Diagrama de Caja de {col}')
            axes[1].set_ylabel(col)
            
            plt.tight_layout()
            plt.savefig(f"{output_dir}/{col}_analisis_numerico.png")
            plt.close()

def analisis_variables_categoricas(data, guardar_graficos=True):
    """Analiza variables categóricas con visualizaciones mejoradas"""
    # Seleccionar columnas categóricas (incluyendo bool y object)
    columnas_categoricas = data.select_dtypes(include=['object', 'bool', 'category']).columns
    
    print(f"\nAnalizando {len(columnas_categoricas)} variables categóricas...")
    
    for col in columnas_categoricas:
        # Saltar columnas con demasiados valores únicos
        if data[col].nunique() > 20:
            print(f"Saltando {col} - demasiados valores únicos ({data[col].nunique()})")
            continue
            
        print(f"\nAnálisis para {col}:")
        
        # Conteo de valores y porcentajes
        conteo_valores = data[col].value_counts()
        porcentaje_valores = data[col].value_counts(normalize=True) * 100
        
        print(f"Valores únicos: {data[col].nunique()}")
        for valor, conteo in conteo_valores.items():
            print(f"  {valor}: {conteo} ({porcentaje_valores[valor]:.2f}%)")
        
        if guardar_graficos:
            plt.figure(figsize=(12, 6))
            
            # Gráfico de barras con porcentajes
            ax = sns.countplot(data=data, x=col, order=conteo_valores.index)
            ax.set_title(f'Distribución de {col}')
            ax.set_xlabel(col)
            ax.set_ylabel('Conteo')
            
            # Añadir etiquetas de porcentaje
            total = len(data[col])
            for p in ax.patches:
                porcentaje = f'{100 * p.get_height() / total:.1f}%'
                x = p.get_x() + p.get_width() / 2
                y = p.get_height()
                ax.annotate(porcentaje, (x, y), ha='center', va='bottom')
            
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.savefig(f"{output_dir}/{col}_analisis_categorico.png")
            plt.close()

# 5. Análisis bivariante
def analisis_bivariante(data, columna_objetivo=None, guardar_graficos=True):
    """Realiza análisis bivariante entre características y opcionalmente una variable objetivo"""
    print("\n=== ANÁLISIS BIVARIANTE ===")
    
    # Matriz de correlación para variables numéricas
    columnas_numericas = data.select_dtypes(include=[np.number]).columns
    
    if len(columnas_numericas) > 1:
        print("\nAnálisis de Correlación:")
        matriz_corr = data[columnas_numericas].corr().round(2)
        
        # Encontrar las correlaciones más fuertes (excluyendo auto-correlaciones)
        correlaciones_fuertes = []
        for i in range(len(matriz_corr.columns)):
            for j in range(i):
                if abs(matriz_corr.iloc[i, j]) >= 0.5:  # Umbral para correlación fuerte
                    correlaciones_fuertes.append((matriz_corr.columns[i], matriz_corr.columns[j], matriz_corr.iloc[i, j]))
        
        # Imprimir correlaciones fuertes
        if correlaciones_fuertes:
            print("\nCorrelaciones fuertes (|r| >= 0.5):")
            for var1, var2, corr in sorted(correlaciones_fuertes, key=lambda x: abs(x[2]), reverse=True):
                print(f"  {var1} y {var2}: {corr:.2f}")
        
        if guardar_graficos:
            # Mapa de calor
            plt.figure(figsize=(12, 10))
            mask = np.triu(np.ones_like(matriz_corr, dtype=bool))
            sns.heatmap(matriz_corr, annot=True, cmap='coolwarm', mask=mask, 
                        linewidths=0.5, fmt='.2f', cbar_kws={'label': 'Coeficiente de Correlación'})
            plt.title('Matriz de Correlación')
            plt.tight_layout()
            plt.savefig(f"{output_dir}/matriz_correlacion.png")
            plt.close()
            
            # Matriz de gráficos de dispersión para características altamente correlacionadas
            if correlaciones_fuertes:
                vars_alta_corr = list(set([item[0] for item in correlaciones_fuertes] + [item[1] for item in correlaciones_fuertes]))
                if len(vars_alta_corr) <= 6:  # Limitar para evitar demasiados gráficos
                    sns.pairplot(data[vars_alta_corr], height=2.5, corner=True)
                    plt.suptitle('Matriz de Gráficos de Dispersión para Variables Altamente Correlacionadas', y=1.02)
                    plt.savefig(f"{output_dir}/matriz_dispersion_alta_corr.png")
                    plt.close()
    
    # Analizar relaciones entre variables categóricas y numéricas
    if guardar_graficos:
        columnas_categoricas = data.select_dtypes(include=['object', 'bool', 'category']).columns
        
        # Limitar a categorías con menos de 8 valores únicos para evitar gráficos sobrecargados
        columnas_categoricas = [col for col in columnas_categoricas if data[col].nunique() < 8]
        
        # Si se especifica una variable objetivo y es numérica
        if columna_objetivo and columna_objetivo in columnas_numericas:
            print(f"\nAnalizando variables categóricas contra objetivo: {columna_objetivo}")
            
            for col_cat in columnas_categoricas:
                plt.figure(figsize=(12, 6))
                sns.boxplot(data=data, x=col_cat, y=columna_objetivo)
                plt.title(f'{col_cat} vs {columna_objetivo}')
                plt.xticks(rotation=45, ha='right')
                plt.tight_layout()
                plt.savefig(f"{output_dir}/{col_cat}_vs_{columna_objetivo}.png")
                plt.close()
                
                # Análisis de valores por grupo
                grupos_stats = data.groupby(col_cat)[columna_objetivo].agg(['mean', 'median', 'std', 'count'])
                print(f"\n  {col_cat} vs {columna_objetivo} - Estadísticas por grupo:")
                print(grupos_stats)
                
                # Comprobación simple de varianza entre grupos
                min_media = grupos_stats['mean'].min()
                max_media = grupos_stats['mean'].max()
                if max_media - min_media > data[columna_objetivo].std():
                    print(f"  * Posible relación significativa detectada (gran diferencia entre grupos)")

# 6. Análisis específico para marketing
def analisis_marketing(data, guardar_graficos=True):
    """Realiza análisis específicos para campañas de marketing"""
    print("\n=== ANÁLISIS DE CAMPAÑA DE MARKETING ===")
    
    # Comprobar si existen métricas clave de marketing en el dataset
    metricas_clave = ['ROI', 'Conversion_Rate', 'Campaign_Type', 'Budget', 'Revenue', 'Cost']
    metricas_existentes = [metrica for metrica in metricas_clave if metrica in data.columns]
    
    if not metricas_existentes:
        print("No se encontraron métricas estándar de marketing en el dataset.")
        return
    
    print(f"Analizando métricas clave de marketing: {', '.join(metricas_existentes)}")
    
    # Análisis de ROI por tipo de campaña si ambas columnas existen
    if 'ROI' in data.columns and 'Campaign_Type' in data.columns:
        print("\nAnálisis de ROI por Tipo de Campaña:")
        roi_por_campania = data.groupby('Campaign_Type')['ROI'].agg(['mean', 'median', 'std', 'count'])
        print(roi_por_campania.sort_values(by='mean', ascending=False))
        
        if guardar_graficos:
            plt.figure(figsize=(12, 6))
            sns.boxplot(data=data, x='Campaign_Type', y='ROI')
            plt.title('ROI por Tipo de Campaña')
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            plt.savefig(f"{output_dir}/roi_por_tipo_campania.png")
            plt.close()
    
    # Análisis de tasa de conversión
    if 'Conversion_Rate' in data.columns:
        print("\nAnálisis de Tasa de Conversión:")
        conv_stats = data['Conversion_Rate'].describe()
        print(conv_stats)
        
        # Analizar por variables categóricas si existen
        for col_cat in ['Location', 'Channel', 'Audience', 'Campaign_Type']:
            if col_cat in data.columns:
                print(f"\nTasa de Conversión por {col_cat}:")
                conv_por_cat = data.groupby(col_cat)['Conversion_Rate'].agg(['mean', 'median', 'std', 'count'])
                print(conv_por_cat.sort_values(by='mean', ascending=False))
                
                if guardar_graficos:
                    plt.figure(figsize=(12, 6))
                    sns.barplot(data=data, x=col_cat, y='Conversion_Rate', ci=None)
                    plt.title(f'Tasa de Conversión Promedio por {col_cat}')
                    plt.xticks(rotation=45, ha='right')
                    plt.tight_layout()
                    plt.savefig(f"{output_dir}/conversion_por_{col_cat}.png")
                    plt.close()
    
    # Análisis de eficiencia del presupuesto
    if 'Budget' in data.columns and 'Revenue' in data.columns:
        print("\nAnálisis de Eficiencia del Presupuesto:")
        data['ROI_Calculado'] = (data['Revenue'] - data['Budget']) / data['Budget']
        print(data[['Budget', 'Revenue', 'ROI_Calculado']].describe())
        
        if guardar_graficos:
            plt.figure(figsize=(10, 8))
            sns.scatterplot(data=data, x='Budget', y='Revenue')
            plt.title('Presupuesto vs Ingresos')
            plt.tight_layout()
            plt.savefig(f"{output_dir}/presupuesto_vs_ingresos.png")
            plt.close()

# 7. Ejecución principal
if __name__ == "__main__":
    print("Iniciando Análisis Exploratorio de Datos...")
    
    # Ejecutar análisis
    analisis_variables_numericas(datos)
    analisis_variables_categoricas(datos)
    
    # Intentar identificar una posible variable objetivo (común en datasets de marketing)
    posibles_objetivos = ['ROI', 'Conversion_Rate', 'Revenue', 'Sales']
    variable_objetivo = None
    for objetivo in posibles_objetivos:
        if objetivo in datos.columns:
            variable_objetivo = objetivo
            print(f"\nSe identificó {objetivo} como posible variable objetivo para análisis")
            break
    
    analisis_bivariante(datos, columna_objetivo=variable_objetivo)
    analisis_marketing(datos)
    
    # 8. Generar informe completo
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    nombre_informe = f"informe_eda_marketing_{timestamp}.txt"
    
    print(f"\nGenerando informe completo: {nombre_informe}")
    
    with open(nombre_informe, 'w') as f:
        f.write("=====================================================\n")
        f.write("    ANÁLISIS EXPLORATORIO DE DATOS DE MARKETING     \n")
        f.write("=====================================================\n\n")
        f.write(f"Fecha de Análisis: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
        
        f.write("1. VISIÓN GENERAL DEL DATASET\n")
        f.write("----------------------------\n")
        f.write(f"Filas: {datos.shape[0]}\n")
        f.write(f"Columnas: {datos.shape[1]}\n")
        f.write(f"Uso de Memoria: {datos.memory_usage(deep=True).sum() / (1024**2):.2f} MB\n\n")
        
        f.write("2. INFORMACIÓN DE COLUMNAS\n")
        f.write("-------------------------\n")
        # Usar StringIO estándar en lugar de pd.io.StringIO
        from io import StringIO
        buffer = StringIO()
        datos.info(buf=buffer)
        f.write(buffer.getvalue())
        f.write("\n\n")
        
        f.write("3. ESTADÍSTICAS DESCRIPTIVAS\n")
        f.write("---------------------------\n")
        f.write(datos.describe().round(2).to_string())
        f.write("\n\n")
        
        f.write("4. VALORES FALTANTES\n")
        f.write("-------------------\n")
        missing_data = pd.DataFrame({
            'Conteo': datos.isnull().sum(),
            'Porcentaje': (datos.isnull().mean() * 100).round(2)
        })
        missing_data = missing_data[missing_data['Conteo'] > 0]
        if not missing_data.empty:
            f.write(missing_data.to_string())
        else:
            f.write("No se encontraron valores faltantes en el dataset.")
        f.write("\n\n")
        
        if variable_objetivo:
            f.write(f"5. VARIABLE OBJETIVO: {variable_objetivo}\n")
            f.write("-" * (22 + len(variable_objetivo)) + "\n")
            f.write(f"Estadísticas básicas para {variable_objetivo}:\n")
            f.write(datos[variable_objetivo].describe().to_string())
            f.write("\n\n")
        
        f.write("6. HALLAZGOS CLAVE\n")
        f.write("------------------\n")
        f.write("* El análisis EDA está completo. Gráficos disponibles en 'eda_plots'.\n")
              if 'Campaign_Type' in datos.columns and 'ROI' in datos.columns:
            top_campaign = datos.groupby('Campaign_Type')['ROI'].mean().sort_values(ascending=False).index[0]
            f.write(f"* El tipo de campaña con el ROI promedio más alto es '{top_campaign}'.\n")
        f.write("\n")
        
    print(f"\n¡EDA completado! Informe guardado como '{nombre_informe}'")
    print(f"Visualizaciones guardadas en el directorio '{output_dir}'")

Cargando dataset...
Dataset cargado exitosamente con 200000 filas y 16 columnas.

=== VISIÓN GENERAL DE LOS DATOS ===

Primeras 5 filas del dataset:
   Campaign_ID              Company Campaign_Type Target_Audience Duration  \
0            1  Innovate Industries         Email       Men 18-24  30 days   
1            2       NexGen Systems         Email     Women 35-44  60 days   
2            3    Alpha Innovations    Influencer       Men 25-34  30 days   
3            4   DataTech Solutions       Display        All Ages  60 days   
4            5       NexGen Systems         Email       Men 25-34  15 days   

  Channel_Used  Conversion_Rate Acquisition_Cost   ROI     Location  Language  \
0   Google Ads             0.04       $16,174.00  6.29      Chicago   Spanish   
1   Google Ads             0.12       $11,566.00  5.61     New York    German   
2      YouTube             0.07       $10,200.00  7.18  Los Angeles    French   
3      YouTube             0.11       $12,724.00  5.55    

C:\Users\gequirog\AppData\Local\Temp\ipykernel_11976\698945970.py:282: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar=None` for the same effect.

  sns.barplot(data=data, x=col_cat, y='Conversion_Rate', ci=None)



Tasa de Conversión por Campaign_Type:
                   mean  median       std  count
Campaign_Type                                   
Influencer     0.080315    0.08  0.040454  40169
Social Media   0.080135    0.08  0.040523  39817
Display        0.080089    0.08  0.040757  39987
Search         0.080021    0.08  0.040544  40157
Email          0.079788    0.08  0.040730  39870


C:\Users\gequirog\AppData\Local\Temp\ipykernel_11976\698945970.py:282: FutureWarning: 

The `ci` parameter is deprecated. Use `errorbar=None` for the same effect.

  sns.barplot(data=data, x=col_cat, y='Conversion_Rate', ci=None)



Generando informe completo: informe_eda_marketing_2025-05-04_21-42-29.txt

¡EDA completado! Informe guardado como 'informe_eda_marketing_2025-05-04_21-42-29.txt'
Visualizaciones guardadas en el directorio 'eda_plots'
